# 🔬 QML Visual Forensics Lab

Interactive notebook for analyzing trades, visualizing patterns, and inspecting ML features.

---

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
from pathlib import Path

# QML imports
from src.reporting.storage import ExperimentLogger
from src.analysis.visualizer import TradeVisualizer

# Initialize
logger = ExperimentLogger()
viz = TradeVisualizer()

print("✅ Lab initialized")

## 1. Load Latest Backtest

In [ ]:
# Get recent runs
recent_runs = logger.get_recent_runs(limit=5)

print("Recent Experiments:")
for run in recent_runs:
    print(f"  {run['run_id']}: {run['strategy_name']} | "
          f"PnL: {run['pnl_percent']:+.2f}% | "
          f"Trades: {run['total_trades']}")

In [ ]:
# Select a run to analyze
RUN_ID = recent_runs[0]['run_id'] if recent_runs else None
print(f"Selected run: {RUN_ID}")

# Load trades
import json
run_data = logger.get_run(RUN_ID)
config = json.loads(run_data['config_json'])

strategy = config.get('detector_method', 'atr')
trades_path = Path(f'../results/{strategy}/{RUN_ID}_trades.csv')

if trades_path.exists():
    trades_df = pd.read_csv(trades_path)
    print(f"Loaded {len(trades_df)} trades")
    display(trades_df.head())
else:
    print(f"Trades file not found: {trades_path}")

In [ ]:
# Load OHLCV data
timeframe = config.get('timeframe', '4h')
data_path = Path(f'../data/processed/BTC/{timeframe}_master.parquet')

df = pd.read_parquet(data_path)
print(f"Loaded {len(df)} candles: {df['time'].min()} to {df['time'].max()}")

## 2. Visualize Individual Trades

Use the TradeVisualizer to inspect specific trades.

In [ ]:
# Plot first trade
TRADE_INDEX = 0

trade = trades_df.iloc[TRADE_INDEX].to_dict()
fig = viz.plot_trade(trade, df, context_bars=50)
fig.show()

In [ ]:
# Plot a winning trade
winning_trades = trades_df[trades_df['result'] == 'WIN']

if len(winning_trades) > 0:
    win_trade = winning_trades.iloc[0].to_dict()
    fig = viz.plot_trade(win_trade, df, context_bars=50)
    fig.show()
else:
    print("No winning trades found")

In [ ]:
# Plot a losing trade for comparison
losing_trades = trades_df[trades_df['result'] == 'LOSS']

if len(losing_trades) > 0:
    loss_trade = losing_trades.iloc[0].to_dict()
    fig = viz.plot_trade(loss_trade, df, context_bars=50)
    fig.show()
else:
    print("No losing trades found")

## 3. ML Feature Analysis

Analyze which features are most predictive of trade outcomes.

In [ ]:
# Load or train XGBoost model
from src.ml.predictor import XGBoostPredictor

model_path = Path('../results/models/xgb_latest.json')

predictor = XGBoostPredictor()

if model_path.exists():
    predictor.load(str(model_path))
    print("Loaded saved model")
else:
    print("Training new model...")
    X, y = predictor.prepare_data(trades_df)
    metrics = predictor.train(X, y)
    print(f"Accuracy: {metrics['accuracy']:.2%}, AUC: {metrics['auc']:.3f}")

In [ ]:
# Top 5 Features
print("\n🏆 Top 5 Most Important Features:")
print("-" * 40)

for i, (feature, importance) in enumerate(predictor.get_top_features(5), 1):
    bar = '█' * int(importance * 50)
    print(f"{i}. {feature:25s} {importance:.4f} {bar}")

In [ ]:
# Feature importance plot
import plotly.express as px

if predictor.feature_importance:
    fi_df = pd.DataFrame([
        {'feature': k, 'importance': v}
        for k, v in predictor.feature_importance.items()
    ]).sort_values('importance', ascending=True)
    
    fig = px.bar(
        fi_df, 
        x='importance', 
        y='feature', 
        orientation='h',
        title='XGBoost Feature Importance'
    )
    fig.show()

## 4. Trade Distribution Analysis

In [ ]:
# Win/Loss distribution
import plotly.express as px

fig = px.histogram(
    trades_df, 
    x='pnl_pct', 
    color='result',
    title='P&L Distribution by Result',
    color_discrete_map={'WIN': '#27ae60', 'LOSS': '#e74c3c'}
)
fig.show()

In [ ]:
# Trade summary stats
print("\n📊 Trade Statistics:")
print("-" * 40)
print(f"Total Trades: {len(trades_df)}")
print(f"Win Rate: {(trades_df['result'] == 'WIN').mean():.1%}")
print(f"Avg Win: {trades_df[trades_df['result'] == 'WIN']['pnl_pct'].mean():.2f}%")
print(f"Avg Loss: {trades_df[trades_df['result'] == 'LOSS']['pnl_pct'].mean():.2f}%")
print(f"Total P&L: {trades_df['pnl_pct'].sum():.2f}%")

---

## Quick Reference

```python
# Plot any trade by index
viz.plot_trade(trades_df.iloc[42].to_dict(), df).show()

# Get prediction for a new signal
signal = {'price': 50000, 'stop_loss': 49000, 'take_profit': 53000, 'validity_score': 0.8}
prob = predictor.predict(signal)
print(f"Win probability: {prob:.1%}")

# Compare grid search results
best_runs = logger.get_top_runs(strategy='grid_search', metric='sharpe_ratio', limit=10)
```